In [1]:
import wallycore as wally

In [2]:
def buildTransaction(tx_inputs, tx_outputs):
    tx = wally.tx_init(2, 0, 1, 2) # version 2, locktime 0, 1 input, 2 outputs

    for tx_input in tx_inputs:
        wally.tx_add_input(tx, tx_input)

    for tx_output in tx_outputs:
        wally.tx_add_output(tx, tx_output)
        
    return tx
    
def buildInput(tx_input_hex, tx_input_vout, sequence):
    tx_input_byte = wally.hex_to_bytes(tx_input_hex)[::-1]
    return wally.tx_elements_input_init(tx_input_byte, tx_input_vout, sequence, None, None, None, None, None, None, None, None, None)

def buildOutput(value, asset_id, script_byte):
    confidential_tx_value_byte = wally.tx_confidential_value_from_satoshi(value)
    return wally.tx_elements_output_init(script_byte, asset_id, confidential_tx_value_byte, None, None, None)

In [3]:
ELEMENTS_UNBLINDED_TX_PREFIX = 0x01    

tx_input_hex = "53174bc15eef5b16522b5fe45cf2e147e829990dd72454ab7d2a493e220e4c36"
asset_id = bytearray([ELEMENTS_UNBLINDED_TX_PREFIX]) + wally.hex_to_bytes("9e56d67dee0fbe395374572d2a389e56659380e2b58ad43081e0d3ddcab1a0f7")[::-1]
script = wally.hex_to_bytes("76a9144f844a62154987b501dccfd6c504596759e9f24888ac")
sequence = 0xffffffff

tx_inputs = [buildInput(tx_input_hex, 0, sequence)]
tx_outputs = [buildOutput(12345, asset_id, script), buildOutput(500, asset_id, None)]

tx = buildTransaction(tx_inputs, tx_outputs)

tx_hex = wally.tx_to_hex(tx, 0)
print(tx_hex)

020000000001364c0e223e492a7dab5424d70d9929e847e1f25ce45f2b52165bef5ec14b17530000000000ffffffff0201f7a0b1caddd3e08130d48ab5e2809365569e382a2d57745339be0fee7dd6569e010000000000003039001976a9144f844a62154987b501dccfd6c504596759e9f24888ac01f7a0b1caddd3e08130d48ab5e2809365569e382a2d57745339be0fee7dd6569e0100000000000001f4000000000000


In [4]:
def generateRedeemScript(private_keys, threshold):
    public_keys = bytearray([])
    for private_key in private_keys:
        public_key  = wally.ec_public_key_from_private_key(private_key)
        public_keys += public_key
    return wally.scriptpubkey_multisig_from_bytes(public_keys, threshold, 0)

In [5]:
def signInputs(tx, private_keys, input_values, prev_out_scripts, sighashes):  
    for index, (private_keys, input_value, prev_out_script, sighash) in enumerate(zip(private_keys, input_values, prev_out_scripts, sighashes)):
        witness_stack = wally.tx_witness_stack_init(len(private_keys) + 2)
        wally.tx_witness_stack_add_dummy(witness_stack, wally.WALLY_TX_DUMMY_NULL);
        
        input_value = wally.tx_confidential_value_from_satoshi(input_value);
        hash_to_sign = wally.tx_get_elements_signature_hash(tx, index, prev_out_script, input_value, sighash, wally.WALLY_TX_FLAG_USE_WITNESS)
        
        for private_key in private_keys:
            public_key  = wally.ec_public_key_from_private_key(private_key)
            signature = wally.ec_sig_from_bytes(private_key, hash_to_sign, wally.EC_FLAG_ECDSA | wally.EC_FLAG_GRIND_R)
            der_signature = wally.ec_sig_to_der(signature) + bytearray([wally.WALLY_SIGHASH_ALL])
            wally.tx_witness_stack_add(witness_stack, der_signature)
        
        wally.tx_witness_stack_add(witness_stack, prev_out_script)
        script_sig = bytearray([0x22, 0x00, 0x20]) + wally.sha256(prev_out_script)
        wally.tx_set_input_script(tx, index, script_sig)
        wally.tx_set_input_witness(tx, index, witness_stack)


In [6]:
private_keys = [wally.hex_to_bytes("d23388f6ed69564cbd7518bde279bcffac2156aeea024fe527726d7b5250d461"), wally.hex_to_bytes("c1c4cfce079a36bd8d1d37b0342cc16f1bd3e5e29d5750601b27ba5e82cb6741"), wally.hex_to_bytes("2554c3ee6bea1073cfe6b9cd94328eb6c7ac510af67c1371f7e379563e02303c")]
redeem_script = generateRedeemScript(private_keys, 2)
signInputs(tx, [private_keys[:2]], [12845], [redeem_script], [wally.WALLY_SIGHASH_ALL])

tx_hex = wally.tx_to_hex(tx, wally.WALLY_TX_FLAG_USE_WITNESS)
print(tx_hex)

020000000101364c0e223e492a7dab5424d70d9929e847e1f25ce45f2b52165bef5ec14b175300000000232200204773ea668c1f16dffbaa5bc11e351f3aa119dc139dc6c1587fa1155e331103d6ffffffff0201f7a0b1caddd3e08130d48ab5e2809365569e382a2d57745339be0fee7dd6569e010000000000003039001976a9144f844a62154987b501dccfd6c504596759e9f24888ac01f7a0b1caddd3e08130d48ab5e2809365569e382a2d57745339be0fee7dd6569e0100000000000001f40000000000000000040047304402200592d11c2c7ef665a1ad78696deb7f05668ec33d22f1a68c4339bb77b51f2b1002200dc973d237df49ce3764d715264bd21682917d249062e1a5ccb685e6d56371f301473044022076a9968f665f075ec63e02114d653d3d5396e37048df6df027d631990e1984ce02201600fa00a73e15190c2b9715369bbb23e2c75c979080872c4607379e48593adc01695221037a840b5e5d903093671dca125b0bfde0a784a44ccb990be90f680fa14460e52d2103b5c9d5b8a930f78780b614d53db72ba4798d2cd6cdeb252f9a6347bf6c680e372102b453acd83aa9b9e29e5ec64e3283b189e84f8faa20dd5fb553ee41b7218d24f953ae0000000000
